In [1]:
library(twang)
library(weights)
library(Matching)
library(tidyverse)

Loading required package: gbm
Loading required package: survival
Loading required package: lattice
Loading required package: splines
Loading required package: parallel
Loaded gbm 2.1.3
Loading required package: survey
Loading required package: grid
Loading required package: Matrix

Attaching package: ‘survey’

The following object is masked from ‘package:graphics’:

    dotchart

Loading required package: xtable
Loading required package: latticeExtra
Loading required package: RColorBrewer
Loading required package: Hmisc
Loading required package: Formula
Loading required package: ggplot2

Attaching package: ‘ggplot2’

The following object is masked from ‘package:latticeExtra’:

    layer


Attaching package: ‘Hmisc’

The following objects are masked from ‘package:xtable’:

    label, label<-

The following object is masked from ‘package:survey’:

    deff

The following objects are masked from ‘package:base’:

    format.pval, units

Loading required package: gdata
gdata: read.xls suppo

In [2]:
data_dir <- file.path("..", "data")

In [3]:
full_data <- readRDS(file.path(data_dir, "full_data_ps.rds"))
head(full_data)

icustay_id,hadm_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,lab_lactate_flag,lab_lactate_first,lab_lactate_min,lab_lactate_max,lab_lactate_abnormal,sedative,echo_int,mort_28_day_int,ps,ps_weight
228416,159895,5491,MICU,2151-06-29 01:18:27,2151-07-27 16:00:56,1,76.58225,1,2151-06-30,⋯,1,1.4,1.4,1.4,0,0,1,0,0.5444486,1.836721
278148,169700,49395,MICU,2197-01-09 00:45:42,2197-01-10 06:46:29,1,82.72606,1,NA,⋯,0,NA,NA,NA,NA,0,0,1,0.2569337,1.345775
274174,103722,14855,MICU,2181-09-08 23:43:13,2181-09-26 16:52:02,1,61.84390,1,2181-09-11,⋯,1,1.6,1.6,2.6,1,0,1,0,0.3264142,3.063592
214954,170515,54642,MICU,2176-03-11 04:58:23,2176-03-15 18:00:48,1,63.86507,1,2176-03-11,⋯,1,3.8,2.2,3.8,1,1,1,0,0.8063102,1.240217
222457,183493,96815,MICU,2105-06-04 20:57:55,2105-06-06 20:01:51,1,62.84018,1,NA,⋯,0,NA,NA,NA,NA,0,0,0,0.2239520,1.288580
264413,134244,81436,MICU,2133-03-27 16:31:37,2133-04-04 16:15:28,1,62.60148,1,2133-03-31,⋯,1,1.4,1.4,1.4,0,0,1,0,0.3457691,2.892104


In [4]:
library(tableone)

In [5]:
features <- c("age", "gender", "first_careunit", "weight", "elix_score",
              "saps", "sofa", "vent", "vaso", "sedative",
              "icd_chf", "icd_afib", "icd_renal", "icd_liver",
              "icd_copd", "icd_cad", "icd_stroke", "icd_malignancy", "icu_adm_weekday",
              "vs_map_first", "vs_heart_rate_first", "vs_temp_first", "vs_cvp_first",
              "lab_wbc_first", "lab_hemoglobin_first", "lab_platelet_first",
              "lab_sodium_first", "lab_potassium_first", "lab_bicarbonate_first",
              "lab_chloride_first", "lab_bun_first", "lab_lactate_first",
              "lab_creatinine_first", "lab_ph_first", "lab_po2_first", "lab_pco2_first",
              "lab_bnp_flag", "lab_troponin_flag", "lab_creatinine_kinase_flag")

In [6]:
tab <- CreateTableOne(vars = features,
                      argsNormal = list(var.equal = FALSE),
                      strata = "echo",
                      data = full_data,)
capture.output(tab_df <- tab %>% print(smd = TRUE) %>% as.data.frame(stringsAsFactors = FALSE)) %>% invisible

In [7]:
tab_df

,0,1,p,test,SMD
n,3099,3262,,,
age (mean (sd)),66.69 (17.21),65.74 (16.55),0.026,,0.056
gender = F (%),1575 (50.8),1558 (47.8),0.016,,0.061
first_careunit = MICU (%),2404 (77.6),2593 (79.5),0.067,,0.047
weight (mean (sd)),78.56 (23.58),83.17 (26.87),<0.001,,0.182
elix_score (mean (sd)),8.51 (7.45),10.07 (7.67),<0.001,,0.207
saps (mean (sd)),19.63 (5.79),20.76 (5.44),<0.001,,0.202
sofa (mean (sd)),5.31 (3.62),6.33 (3.79),<0.001,,0.277
vent = 1 (%),1459 (47.1),1901 (58.3),<0.001,,0.226
vaso = 1 (%),839 (27.1),1228 (37.6),<0.001,,0.227


In [8]:
ipw_svydesign <- readRDS(file.path(data_dir, "ipw_svydesign.rds"))

In [9]:
tab_wtd <- svyCreateTableOne(vars = features, strata = "echo", data = ipw_svydesign)

In [10]:
capture.output(tab_wtd_df <- tab_wtd %>% print %>% as.data.frame(stringsAsFactors = FALSE)) %>% invisible

In [11]:
tab_wtd_df

,0,1,p,test
n,5739.99,5845.39,,
age (mean (sd)),66.10 (16.23),65.18 (16.70),0.062,
gender = F (%),2828.1 (49.3),2822.6 (48.3),0.471,
first_careunit = MICU (%),4474.1 (77.9),4648.2 (79.5),0.154,
weight (mean (sd)),82.38 (23.55),83.44 (25.27),0.164,
elix_score (mean (sd)),10.17 (7.45),10.04 (7.81),0.098,
saps (mean (sd)),23.12 (5.60),23.42 (5.17),0.127,
sofa (mean (sd)),8.26 (4.26),8.51 (3.79),0.036,
vent = 1 (%),2904.8 (50.6),3135.1 (53.6),0.027,
vaso = 1 (%),1753.8 (30.6),1966.2 (33.6),0.016,


In [12]:
ps_icustay_id <- data.table::fread(file.path(data_dir, "ps_icustay_id"), data.table = FALSE) %>% pull(icustay_id)

In [13]:
ps_df <- full_data %>% filter(icustay_id %in% ps_icustay_id)

In [14]:
tab_ps <- CreateTableOne(vars = features,
                         strata = "echo",
                         argsNormal = list(var.equal = FALSE),
                         data = ps_df)

In [15]:
capture.output(tab_ps_df <- tab_ps %>% print(smd = TRUE) %>% as.data.frame(stringsAsFactors = FALSE)) %>% invisible

In [16]:
tab_ps_df

,0,1,p,test,SMD
n,1629,1629,,,
age (mean (sd)),66.63 (16.51),65.85 (16.95),0.180,,0.047
gender = F (%),776 (47.6),791 (48.6),0.623,,0.018
first_careunit = MICU (%),1265 (77.7),1289 (79.1),0.328,,0.036
weight (mean (sd)),81.50 (24.98),79.27 (22.67),0.011,,0.093
elix_score (mean (sd)),9.57 (7.69),8.95 (7.63),0.021,,0.081
saps (mean (sd)),20.35 (5.41),19.75 (5.14),0.001,,0.115
sofa (mean (sd)),6.00 (3.75),5.53 (3.47),<0.001,,0.129
vent = 1 (%),880 (54.0),804 (49.4),0.009,,0.093
vaso = 1 (%),524 (32.2),487 (29.9),0.173,,0.049
